In [7]:
import urllib
import urllib.request
import urllib.parse
#import tldextract
import re
from bs4 import BeautifulSoup

#=== Read Excel Sheet =========
import xlrd

#=== Write to CSV file ===
import csv

#==Os library baig used to delete a file
import os

#urlResultSheetPath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping\Website_Extraction_results.csv"
urlResultSheetPath=r"C:\Users\gaurav.anand\Downloads\Web Scraping AI\Website_Extraction_results.csv"

def delete_extraction_result_File():
    global urlResultSheetPath
    #urlResultSheetPath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping\Website_Extraction_results.csv"
    if os.path.exists(urlResultSheetPath):
        os.remove(urlResultSheetPath)
        print("Result File deleted : " + urlResultSheetPath)
    
#zero based Index and Column
def read_excel_sheet(sheetIndex, rowNum, colNum):
    
    #urlSheetPath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping\Websites for Charles.xlsx"
    urlSheetPath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping\Websites-Main 8.xlsx"
    Web_Sites=[] #Holds thr tickets

    
    # To open Workbook 
    wb = xlrd.open_workbook(urlSheetPath) 
    sheet = wb.sheet_by_index(sheetIndex) 

    print("Number of Web Sites :",sheet.nrows)

    for i in range(rowNum,sheet.nrows):
        Web_Sites.append(sheet.cell_value(i, colNum))
       # print(sheet.cell_value(i, 0),' ',sheet.cell_value(i, 3))
    
    return Web_Sites

def Addto_extrction_result_sheet(url,status):
    
    mydata=[]
    global urlResultSheetPath
    #urlResultSheetPath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping\Website_Extraction_results.csv"
    
    myFile = open(urlResultSheetPath, 'a') 
    with myFile:
        writer = csv.writer(myFile,delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([url,status])
        
    

#Removes all the special charachers in string/Website URL
def strip_url(url):
    return re.sub('\W+','', url)
    
def write_Website_text(webText,url):
    #dbFilePath="C:\\Users\\gaurav.anand\\OneDrive - NIIT Technologies 1\\Automation COC\\SVN\\Assisted RCA\\LastSych.txt"
    #dbFilePath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\Assisted RCA\LastSych.txt"
    print("Writing file for "+ url + " to Disk ")
    filename= strip_url(url)
    dbFilePath=r"C:\Users\gaurav.anand\OneDrive - NIIT Technologies 1\Automation COC\SVN\WebScraping\WebScraping"
    dbFilePath+="\\"+filename +".txt" 
    
    webSiteTextFile = open(dbFilePath,"w")
    webSiteTextFile.write(webText)  
    print("Web Site Content for " + url +" has been written to Disk File")
    webSiteTextFile.close()

#url="https://www.1620ia.com/our-people"
#url="http://www.1650wealth.com/"

#url="https://www.1776wealth.com/"
#url="https://www.1776wealth.com/contact"
#url="https://www.1776wealth.com/about-us/bio?_=sean"
#url="http://www.1650wealth.com/who-we-are/our-team/"



#List of Test URL's
urlArr=[]
urlArr.append("http://www.3rdcreek.com")
#urlArr.append("http://www.3rdcreek.com/investments-management-team/")
#urlArr.append("https://3dwealthadvisors.com/contact-us/")
urlArr.append("http://12-squared.com")
#urlArr.append("https://www.1620ia.com/our-people")
urlArr.append("http://www.1650wealth.com/")
urlArr.append("https://www.1776wealth.com/")
#urlArr.append("https://www.1776wealth.com/contact")
#urlArr.append("https://www.1776wealth.com/about-us/bio?_=sean")
#urlArr.append("http://www.1650wealth.com/who-we-are/our-team/")
urlArr.append("http://www.3rdstreetcap.com")
#urlArr.append("http://www.3rdstreetcap.com/contact.html")



#Hint List of Sublinks
hintUrlLinks=[]
hintUrlLinks.append("about us")
hintUrlLinks.append("about-us")
hintUrlLinks.append("team")
hintUrlLinks.append("leadership")
hintUrlLinks.append("contact")
hintUrlLinks.append("people")
hintUrlLinks.append("story")


#List of Website Sublinks
websiteSubLinks=[]

#Checks if webste is relevant for content scraping
def isRelevantSubsite(subsiteUrl, hrefText):
    isRelevantSubsite=None
    
    matches = (x for x in hintUrlLinks if x in hrefText or x in subsiteUrl)
    for x in matches:
        isRelevantSubsite=True
        print("===>>>>>>>    =====<<<<<<")
        print("\n" +" Subsite : " +subsiteUrl +" is relevant for scraping for keyword : " +  str(x))
        print("===>>>>>>>    =====<<<<<<")
    
    return isRelevantSubsite

def addRelevantSubsitetoList(domainUrl,subsiteURL):
    
    global  websiteSubLinks
    
    #print('Pre --->Subsite being Added to List for extraction : '+ subsiteURL)
    if  subsiteURL.find("http")==-1:
        if subsiteURL[0]!=r"/":
            subsiteURL="/"+subsiteURL
        subsiteURL=domainUrl+subsiteURL
   
    #print('-->Subsite being Added to List for extraction : '+ subsiteURL)
    if subsiteURL not in websiteSubLinks:
        websiteSubLinks.append(subsiteURL)
        #print('Subsite Added to List for extraction : '+ subsiteURL)
    
def remove_NewLine_Characters_fromWebText(webText):
    webText=webText.replace('\n',' ').replace('\r','')
    return webText
    
def is_ExcludeLine(lineString):
    isBlackListed=False
    #print(" =>>> Line Strinig before Black Listing : " + lineString)
    blacklist = [".html", "©","facebook","youtube","twitter","instagram","linkedin"]
    for obj in blacklist:
       
        if obj in lineString:
            print("Black List Object : " + obj)
            isBlackListed=True
                
    return isBlackListed
        
    
def extract_website_content(url,isSubsite=False):
    if not isSubsite:
        websiteSubLinks.clear()
   
    httpIndex= url.find("http")
    print("Http Index in URL " + url + " is : " + str(httpIndex))
    if httpIndex== -1:
        
        url="http://"+url
        
    print("Extracting Page URL  : " + url) 
    print("Is Subsite : " + str(isSubsite)) 
    
    try:
        
        thePage=urllib.request.urlopen(url)
        print('Successfully Opened Url')
    
        
        soup=BeautifulSoup(thePage,"html.parser")

        #blacklist = ["script", "style","img","link" ]
        blacklist = ["script", "style","img" ]
        [s.extract() for s in soup(blacklist)] # remove tags in Blacklist

        webText=""



        for link in soup.findAll():
            #print(link.get('href'))
            #print(link.text)

            #webText +=link.name
            #webText+="\n"
            if not is_ExcludeLine(link.text):
                webText +=link.text
                        
            
            if link.name=="a": # add Leadership/Contect list check
                webText +=" : "
                #webText+=str(link.get('href'))
                href=str(link.get('href'))
                if not is_ExcludeLine(href): #href.find(".html")==-1: 
                    webText+=href

                if not isSubsite and isRelevantSubsite(str(link.get('href')),link.text):
                      addRelevantSubsitetoList(url,str(link.get('href')))

            
            webText+="\n"
            #webText +="======="
            #webText+="\n"


        print ("   ")
        print ("   ")
        print ("====================")
        print (" Printing Webtext  ")
        print ("====================")
        #print (webText)
        #webText=remove_NewLine_Characters_fromWebText(webText)
        write_Website_text(webText,url)
        Addto_extrction_result_sheet(url,"Success")

    
    except Exception as e:
        print('Error Opening url : ', str(e) )
        
        Addto_extrction_result_sheet(url,"Fail")
        pass
        
           



def execute_content_extraction_engine():
   
    global urlArr
    global websiteSubLinks
    
    url_list=[]
    #Delete Extraction Result Sheet
    delete_extraction_result_File()
    
    #Read all 80K domain names from Excel Sheet
    url_list=read_excel_sheet(0,1,0)
    
    #Extract Contant for Main Site
    for urlSite in url_list:
        extract_website_content(urlSite,False)
        #Extract Contant for Subsite
        for urlSubSite in websiteSubLinks:
            extract_website_content(urlSubSite,True)
    
    
    print(websiteSubLinks)

execute_content_extraction_engine()

Number of Web Sites : 10
Http Index in URL www.12-squared.com is : -1
Extracting Page URL  : http://www.12-squared.com
Is Subsite : False
Successfully Opened Url
   
   
 Printing Webtext  
Writing file for http://www.12-squared.com to Disk 
Web Site Content for http://www.12-squared.com has been written to Disk File
Http Index in URL www.1650wealth.com is : -1
Extracting Page URL  : http://www.1650wealth.com
Is Subsite : False
Successfully Opened Url
Black List Object : ©
Black List Object : ©
Black List Object : ©
Black List Object : ©
===>>>>>>>    =====<<<<<<

 Subsite : http://www.1650wealth.com/who-we-are/story/ is relevant for scraping for keyword : story
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : http://www.1650wealth.com/who-we-are/our-team/ is relevant for scraping for keyword : team
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : http://www.1650wealth.com/contact-us/our-offices/ is relevant for scraping for keyword : contact
===>>>>>>>   

Successfully Opened Url
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/meet-our-team-of-experts/ is relevant for scraping for keyword : team
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/our-story/ is relevant for scraping for keyword : story
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/our-story/ is relevant for scraping for keyword : story
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/meet-our-team-of-experts/ is relevant for scraping for keyword : team
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/contact-us/ is relevant for scraping for keyword : contact
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwealthadvisors.com/our-story/ is relevant for scraping for keyword : story
===>>>>>>>    =====<<<<<<
===>>>>>>>    =====<<<<<<

 Subsite : https://3dwea

Successfully Opened Url
   
   
 Printing Webtext  
Writing file for http://www.1776wealth.com/about-us to Disk 
Web Site Content for http://www.1776wealth.com/about-us has been written to Disk File
Http Index in URL http://www.1776wealth.com/about-us/bio?_=sean is : 0
Extracting Page URL  : http://www.1776wealth.com/about-us/bio?_=sean
Is Subsite : True
Successfully Opened Url
   
   
 Printing Webtext  
Writing file for http://www.1776wealth.com/about-us/bio?_=sean to Disk 
Web Site Content for http://www.1776wealth.com/about-us/bio?_=sean has been written to Disk File
Http Index in URL http://www.1776wealth.com/about-us/bio?_=deanna is : 0
Extracting Page URL  : http://www.1776wealth.com/about-us/bio?_=deanna
Is Subsite : True
Successfully Opened Url
   
   
 Printing Webtext  
Writing file for http://www.1776wealth.com/about-us/bio?_=deanna to Disk 
Web Site Content for http://www.1776wealth.com/about-us/bio?_=deanna has been written to Disk File
Http Index in URL http://www.1776we

Successfully Opened Url
   
   
 Printing Webtext  
Writing file for https://1northwealthservices.com/contact-1-north-wealth/ to Disk 
Web Site Content for https://1northwealthservices.com/contact-1-north-wealth/ has been written to Disk File
['https://1northwealthservices.com/1-north-wealth-team/', 'https://1northwealthservices.com/contact-1-north-wealth/']
